In [1]:
import numpy as np
import cv2

# Bilinear interpolation

In [6]:
'''
python implementation of bilinear interpolation
''' 
def bilinear_interpolation(img,out_dim): # input image and output dimension
    src_h, src_w, channel = img.shape
    dst_h, dst_w = out_dim[1], out_dim[0]
    print ("src_h, src_w = ", src_h, src_w)
    print ("dst_h, dst_w = ", dst_h, dst_w)
    if src_h == dst_h and src_w == dst_w:
        return img.copy()
    dst_img = np.zeros((dst_h,dst_w,3),dtype=np.uint8)
    scale_x, scale_y = float(src_w) / dst_w, float(src_h) / dst_h # calculate scaling factor
    for i in range(channel):
        for dst_y in range(dst_h):
            for dst_x in range(dst_w):
 
                # find the origin x and y coordinates of dst image x and y
                # use geometric center symmetry
                # if use direct way, src_x = dst_x * scale_x
                # not necessary, but makes better interpolation
                src_x = (dst_x + 0.5) * scale_x-0.5 # find corresponding src pixel based on dst pixel
                src_y = (dst_y + 0.5) * scale_y-0.5
 
                # find the coordinates of the points which will be used to compute the interpolation
                # not necessary, but prevents errors
                src_x0 = int(np.floor(src_x))     #np.floor()返回不大于输入参数的最大整数。（向下取整）
                src_x1 = min(src_x0 + 1 ,src_w - 1) #if exceeding edges, use the edge value
                src_y0 = int(np.floor(src_y))
                src_y1 = min(src_y0 + 1, src_h - 1)
 
                # calculate the interpolation
                temp0 = (src_x1 - src_x) * img[src_y0,src_x0,i] + (src_x - src_x0) * img[src_y0,src_x1,i]
                temp1 = (src_x1 - src_x) * img[src_y1,src_x0,i] + (src_x - src_x0) * img[src_y1,src_x1,i]
                dst_img[dst_y,dst_x,i] = int((src_y1 - src_y) * temp0 + (src_y - src_y0) * temp1)
 
    return dst_img

In [8]:
if __name__ == '__main__':
    img = cv2.imread('download.jpeg')
    dst = bilinear_interpolation(img,(700,700))
    cv2.imshow('bilinear interp',dst)
    cv2.waitKey()

src_h, src_w =  358 637
dst_h, dst_w =  700 700


# Histogram Equalization

In [9]:
from matplotlib import pyplot as plt

In [21]:
img = cv2.imread('download.jpeg')

# histogram
hist, bins = np.histogram(img.flatten(), bins = 256, range = [0, 256])

# cumulate
cdf = hist.cumsum()
cdf_normalize = cdf / float(cdf.max())

# equalization
img_equal = np.interp(img.flatten(), bins[:-1], cdf_normalize * 255).astype(np.uint8)
img_equal = img_equal.reshape(img.shape) # reshape back to original shape

# show
cv2.imshow('hist equalization', img_equal)
cv2.waitKey()

-1

## Sobel

In [27]:
img = cv2.imread('download.jpeg', 0) # 0 reads the image in grayscale

# sobel
sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = 3)
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = 3)

# convert back to uint8
sobel_x = np.uint8(np.absolute(sobel_x))
sobel_y = np.uint8(np.absolute(sobel_y))
# or
# sobel_x = cv2.convertScaleAbs(x)

# combine the two directions
img_result = cv2.bitwise_or(sobel_x, sobel_y)
# or
# img_result = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)


cv2.imshow('sobel_x',sobel_x)
cv2.imshow('sobel_y',sobel_y)
cv2.imshow('image',img_result)
cv2.waitKey()
cv2.destroyAllWindows()